In [30]:
from pymongo import MongoClient
import math
import numpy as np
import gpxpy.geo
from tqdm import tqdm
from collections import defaultdict
import json
client = MongoClient()
db = client.yelp

In [2]:
print len(list(db.businesses.find({"city": "Las Vegas"}, { "business_id": 1, "latitude": 1, "longitude": 1})))

22892


In [3]:
def get_all_reviews_by_user_for_city(city):
    bizes = list(db.businesses.find({"city": city}, { "business_id": 1, "latitude": 1, "longitude": 1, "categories": 1}))
    biz_ids = [biz['business_id'] for biz in bizes]
    all_reviews = db.reviews.find({"business_id": {"$in": biz_ids}})
    user_reviews = defaultdict(list)
    for review in tqdm(all_reviews):
        rel_biz = filter(lambda biz: biz['business_id'] == review['business_id'], bizes)[0]
        user_reviews[review['user_id']].append((rel_biz["latitude"], rel_biz["longitude"]))
    return user_reviews

In [6]:
henderson_reviews = get_all_reviews_by_user_for_city("Henderson")


0it [00:00, ?it/s]
29it [00:00, 285.50it/s]
70it [00:00, 313.32it/s]
102it [00:01, 84.04it/s]
137it [00:01, 108.59it/s]
172it [00:01, 136.85it/s]
214it [00:01, 171.33it/s]
255it [00:01, 207.21it/s]
296it [00:01, 242.58it/s]
335it [00:01, 272.72it/s]
373it [00:01, 297.07it/s]
415it [00:02, 324.67it/s]
457it [00:02, 348.18it/s]
499it [00:02, 366.93it/s]
540it [00:02, 365.72it/s]
580it [00:02, 374.34it/s]
620it [00:02, 365.61it/s]
658it [00:02, 327.65it/s]
693it [00:02, 302.67it/s]
725it [00:02, 300.92it/s]
757it [00:03, 301.69it/s]
788it [00:03, 303.17it/s]
827it [00:03, 324.31it/s]
865it [00:03, 338.42it/s]
903it [00:03, 348.50it/s]
944it [00:03, 364.15it/s]
982it [00:03, 367.12it/s]
1023it [00:03, 377.05it/s]
1063it [00:03, 381.45it/s]
1103it [00:03, 385.74it/s]
1143it [00:04, 389.62it/s]
1185it [00:04, 396.86it/s]
1226it [00:04, 400.25it/s]
1267it [00:04, 356.17it/s]
1304it [00:04, 311.94it/s]
1337it [00:04, 289.73it/s]
1368it [00:04, 280.51it/s]
1398it [00:04, 266.65it/s]
1428it [00

In [7]:
henderson_reviews

defaultdict(list,
            {u'ue0esSSw4lVFeLtZgX6KbA': [(36.0107952, -115.063704)],
             u'vb7Be9I2G8AGvSJ3Efd0ww': [(36.0603167507, -115.045475596),
              (36.021803963, -115.048688312)],
             u'y79ebH7iZrkxDkOYyxcygw': [(36.0667760026, -115.042471048)],
             u'8RXS7mhIQ7X4wDRzWY586g': [(36.0554537, -115.0414221),
              (36.0258472, -115.0838817),
              (36.062368, -115.0369872),
              (36.0427729342, -115.046223536),
              (36.0182071, -115.0546223),
              (36.0258472, -115.0838817),
              (36.0571009606, -115.046864077)],
             u'B7mqXzrSFKM_Wff2tbBagw': [(35.9928586, -115.1023338),
              (36.017605, -115.1178915),
              (36.066484915, -115.04658856),
              (36.0015149, -115.1072422),
              (36.017521053, -115.11792887),
              (36.0114785, -114.9921795),
              (36.0259925, -115.0856658),
              (36.035444, -114.985432),
              (36.05

In [208]:
def get_average_lat_and_long(set_lat_by_long):
    set_lat_by_long = np.asarray(set_lat_by_long).copy()
    return np.average(set_lat_by_long, axis=0)

def get_max_distacne_from_mid(set_lat_by_long):
    set_lat_by_long = np.asarray(set_lat_by_long)
    mid = get_average_lat_and_long(set_lat_by_long)
    return np.max([gpxpy.geo.haversine_distance(mid[0], mid[1], lat, lon)/1609.34 for lat, lon in set_lat_by_long])


# Example of some BS

In [163]:
blah = [(36.0216351, -115.0490493), (36.0210746347, -115.085787212), (36.0230899638, -115.048998752), (35.9874346, -115.1016046), (36.0224502974, -115.084934682), (36.0613739924, -115.040918917), (36.019323, 115.086769), (36.0217118, -115.0853057), (36.020552, -115.089984), (35.9644522544, -115.170425241), (36.020867, -115.08958), (36.020558, -115.089697), (35.9799764, -115.1004085), (36.0601255, -115.0457496), (36.0209607, -115.09037), (36.0205397, -115.0895425), (36.0696712, -115.0811296), (36.017605, -115.1178915), (36.0484714, -115.0182747), (36.020696, -115.0901928), (36.0224413067, -115.083539126), (36.0118753, -115.0628743), (36.021803963, -115.048688312), (36.0671402448, -115.042851922), (35.9977566, -115.1263719), (36.0280724, -115.0503663)]
mid = get_average_lat_and_long(blah)
blah = np.asarray(blah).copy()
blah

array([[  36.0216351 , -115.0490493 ],
       [  36.02107463, -115.08578721],
       [  36.02308996, -115.04899875],
       [  35.9874346 , -115.1016046 ],
       [  36.0224503 , -115.08493468],
       [  36.06137399, -115.04091892],
       [  36.019323  ,  115.086769  ],
       [  36.0217118 , -115.0853057 ],
       [  36.020552  , -115.089984  ],
       [  35.96445225, -115.17042524],
       [  36.020867  , -115.08958   ],
       [  36.020558  , -115.089697  ],
       [  35.9799764 , -115.1004085 ],
       [  36.0601255 , -115.0457496 ],
       [  36.0209607 , -115.09037   ],
       [  36.0205397 , -115.0895425 ],
       [  36.0696712 , -115.0811296 ],
       [  36.017605  , -115.1178915 ],
       [  36.0484714 , -115.0182747 ],
       [  36.020696  , -115.0901928 ],
       [  36.02244131, -115.08353913],
       [  36.0118753 , -115.0628743 ],
       [  36.02180396, -115.04868831],
       [  36.06714024, -115.04285192],
       [  35.9977566 , -115.1263719 ],
       [  36.0280724 , -1

In [164]:
mid

array([  36.02275609, -106.22683721])

What the crap!???

In [165]:
blah[6] #this is bad...

array([  36.019323,  115.086769])

In [166]:
from sklearn.cluster import KMeans

In [199]:
def get_clusters(data, n_clusters=2):
    kmeans = KMeans(n_clusters=2).fit(data)
    centers = kmeans.cluster_centers_
    to_return = [[] for i in range(len(centers))]
    for x in data:
        dist = np.linalg.norm(centers - x, 1, axis=1)
        index = np.argmin(dist)
        to_return[index].append(x)
    return sorted([np.vstack(i) for i in to_return], key=lambda x: len(x))[::-1]

In [200]:
get_clusters(blah)

[array([[  36.0216351 , -115.0490493 ],
        [  36.02107463, -115.08578721],
        [  36.02308996, -115.04899875],
        [  35.9874346 , -115.1016046 ],
        [  36.0224503 , -115.08493468],
        [  36.06137399, -115.04091892],
        [  36.0217118 , -115.0853057 ],
        [  36.020552  , -115.089984  ],
        [  35.96445225, -115.17042524],
        [  36.020867  , -115.08958   ],
        [  36.020558  , -115.089697  ],
        [  35.9799764 , -115.1004085 ],
        [  36.0601255 , -115.0457496 ],
        [  36.0209607 , -115.09037   ],
        [  36.0205397 , -115.0895425 ],
        [  36.0696712 , -115.0811296 ],
        [  36.017605  , -115.1178915 ],
        [  36.0484714 , -115.0182747 ],
        [  36.020696  , -115.0901928 ],
        [  36.02244131, -115.08353913],
        [  36.0118753 , -115.0628743 ],
        [  36.02180396, -115.04868831],
        [  36.06714024, -115.04285192],
        [  35.9977566 , -115.1263719 ],
        [  36.0280724 , -115.0503663 ]])

In [201]:
for el in get_clusters(blah):
    print get_max_distacne_from_mid(el)

6.49678745326
0.0


This is a hack that will allow us to ignore cases where we get data more than 5 standard diveations our of the data set. 

In [257]:
henderson_radi = []
weights = [] # the more reviews that a user has given the more confident we are that they are part of the competitive region
for i, val in enumerate(henderson_reviews.values()):
    if len(val) > 1:
        dis_from_mid = get_max_distacne_from_mid(val)
        k = 2
        while dis_from_mid > 100:
            val = get_clusters(val, n_clusters=k)[0]
            k += 1
            dis_from_mid = get_max_distacne_from_mid(val)
        henderson_radi.append(dis_from_mid)
        weights.append(len(val))
            
print np.max(henderson_radi)
print len(henderson_radi)

28.8334774116
18592


In [258]:
print np.dot(np.array(weights).T, np.array(henderson_radi))/np.sum(weights)

4.42971874243


## Let's compress the ideas all together

In [1]:
def get_radius_for_city(city, tol=1):
    from pymongo import MongoClient
    import numpy as np
    import gpxpy.geo
    from tqdm import tqdm
    from collections import defaultdict
    client = MongoClient()
    db = client.yelp
    def get_all_reviews_by_user_for_city(city):
        bizes = list(db.businesses.find({"city": city}, { "business_id": 1, "latitude": 1, "longitude": 1, "categories": 1}))
        user_reviews = defaultdict(list)
        for biz in tqdm(bizes):
            biz_id = biz['business_id']
            rel_review = db.reviews.find({"business_id": biz_id})
            for review in list(rel_review):
                user_reviews[review['user_id']].append((biz["latitude"], biz["longitude"]))
        return user_reviews    
    
    def get_average_lat_and_long(set_lat_by_long):
        set_lat_by_long = np.asarray(set_lat_by_long).copy()
        return np.average(set_lat_by_long, axis=0)

    def get_max_distacne_from_mid(set_lat_by_long):
        set_lat_by_long = np.asarray(set_lat_by_long)
        mid = get_average_lat_and_long(set_lat_by_long)
        return np.max([gpxpy.geo.haversine_distance(mid[0], mid[1], lat, lon)/1609.34 for lat, lon in set_lat_by_long])
    
    def get_clusters(data, n_clusters=2):
        kmeans = KMeans(n_clusters=2).fit(data)
        centers = kmeans.cluster_centers_
        to_return = [[] for i in range(len(centers))]
        for x in data:
            dist = np.linalg.norm(centers - x, 1, axis=1)
            index = np.argmin(dist)
            to_return[index].append(x)
        return sorted([np.vstack(i) for i in to_return], key=lambda x: len(x))[::-1]
    
    city_reviews = get_all_reviews_by_user_for_city(city)
    city_radi = []
    review_weights = [] # the more reviews that a user has given the more confident we are that they are part of the competitive region
    for i, val in enumerate(city_reviews.values()):
        if len(val) > 1:
            dis_from_mid = get_max_distacne_from_mid(val)
            k = 2
            while dis_from_mid > 100:
                val = get_clusters(val, n_clusters=k)[0]
                k += 1
                dis_from_mid = get_max_distacne_from_mid(val)
            city_radi.append(dis_from_mid)
            review_weights.append(len(val))
            
    final = np.dot(np.array(review_weights).T, np.array(city_radi))/np.sum(review_weights)
    db.cityDistanceMetric.insert_one({"city" : city, "radius" : final})
    return city, final

In [2]:
print get_radius_for_city("Pheonix")

21it [00:02,  7.95it/s]

8.41463445242


In [5]:
cities = db.businesses.find().distinct("city")
print len(cities)

NameError: name 'db' is not defined

In [269]:
from ipyparallel import Client
import matplotlib.pyplot as plt
client = Client()
dview = client[:]

dview.map_sync(get_radius_for_city, cities)


CompositeError: one or more exceptions from call to method: get_radius_for_city
[0:apply]: NameError: global name 'db' is not defined
[1:apply]: NameError: global name 'db' is not defined
[2:apply]: NameError: global name 'db' is not defined
[3:apply]: NameError: global name 'db' is not defined
.... 4 more exceptions ...